<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textaugment
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [ ]:
import pandas as pd
import numpy as np
from textaugment import Wordnet, EDA, word2vec, Translate
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
import re
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv', 'r') as f:
    df = pd.read_csv(f)

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [ ]:
df.sub_event_type.value_counts()

Peaceful protest                       329399
Armed clash                            239828
Attack                                 132202
Shelling/artillery/missile attack       64743
Mob violence                            42079
Violent demonstration                   41003
Air/drone strike                        34020
Protest with intervention               31784
Remote explosive/landmine/IED           30620
Looting/property destruction            24315
Abduction/forced disappearance          14624
Arrests                                  8668
Change to group/activity                 7925
Disrupted weapons use                    7499
Government regains territory             6148
Other                                    5353
Excessive force against protesters       3098
Grenade                                  2948
Non-state actor overtakes territory      2670
Sexual violence                          2196
Agreement                                1460
Non-violent transfer of territory 

In [ ]:
df.drop_duplicates(['lemmatized']).sub_event_type.value_counts()

Peaceful protest                       305383
Armed clash                             96231
Attack                                  65363
Shelling/artillery/missile attack       33279
Violent demonstration                   24574
Mob violence                            21098
Air/drone strike                        18620
Remote explosive/landmine/IED           16110
Protest with intervention               15003
Looting/property destruction            12036
Abduction/forced disappearance           7214
Change to group/activity                 7098
Arrests                                  4238
Disrupted weapons use                    3866
Other                                    3698
Government regains territory             2424
Grenade                                  1530
Excessive force against protesters       1520
Non-state actor overtakes territory      1198
Sexual violence                          1027
Agreement                                 692
Non-violent transfer of territory 

In [ ]:

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [ ]:
event_types_dict = {}
for num, name in enumerate(list(df['sub_event_type'].unique())):
    event_types_dict[name] = num
    # print(name, " : ", num)

event_types_dict

event_types_dict_inverse = {k:v for v,k in event_types_dict.items()}
print(event_types_dict)
print( event_types_dict_inverse)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [ ]:
def text_to_numerical_category(name):
    num = event_types_dict[name]
    return num

print(text_to_numerical_category('Peaceful protest'))
df['labels'] = df['sub_event_type'].apply(text_to_numerical_category)

1


In [ ]:
df.drop(columns=['data_id', 'event_date', 'year','event_type','region','country', 'location',	'notes','cleaned', 'stemmed'	],inplace=True)

In [ ]:
from sklearn.utils import shuffle

lemm = []
labels = []
THRESH_1 = 20000
THRESH_2 = 10000

for group in df.sub_event_type.unique():
    if len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_1:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))[:THRESH_1])
        labels += ([group]*THRESH_1)

    elif len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_2:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique())))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))))
    else:
        lemm += list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))))


In [ ]:
len(lemm)

246320

In [ ]:
len(labels)

246320

In [ ]:
balanced_df = pd.DataFrame({'Text' : lemm, 'Labels': labels})

In [ ]:
balanced_df.head(n=10)

,Text,Labels
0,houthi artillery fire hit residential neighbor...,Shelling/artillery/missile attack
1,osce smm observed total projectilesexplosions ...,Shelling/artillery/missile attack
2,april regime force shelled arbain village hama...,Shelling/artillery/missile attack
3,prohouthi force reportedly shelled house salim...,Shelling/artillery/missile attack
4,turkish force turkishbacked opposition rebel f...,Shelling/artillery/missile attack
5,military force syria shelled aljisat near kafr...,Shelling/artillery/missile attack
6,regime force shelled balshun town idleb countr...,Shelling/artillery/missile attack
7,november ypg shelled outskirt mare aleppo usin...,Shelling/artillery/missile attack
8,may ukrainian force fired mm mortar position d...,Shelling/artillery/missile attack
9,regime force shelled sarman idleb injuring one...,Shelling/artillery/missile attack


In [ ]:
event_types_dict_new = {}
for num, name in enumerate(list(balanced_df.Labels.unique())):
    event_types_dict_new[name] = num
    # print(name, " : ", num)

event_types_dict_new

event_types_dict_inverse_new = {k:v for v,k in event_types_dict_new.items()}
print(event_types_dict_new)
print( event_types_dict_inverse_new)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [ ]:
balanced_df['NumLabel'] = balanced_df.Labels.apply(text_to_numerical_category)

In [ ]:
ed=balanced_df.head(n=25000)


In [ ]:
ed.iloc(1)[2][0]

0

In [ ]:
sentences = []
for i in range(0,len(ed)):
  sentences.append(ed.iloc(1)[0][i])

len(sentences)

25000

In [ ]:
nums = []
for i in range(0,len(ed)):
  nums.append(ed.iloc(1)[2][i])
len(nums)

25000

In [ ]:

set(stopwords.words('english')) 

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def word_extraction(sentence):    
  ignore = ['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves']    
  words = re.sub("[^\w]", " ",  sentence).split()    
  cleaned_text = [w.lower() 
  for w in words if w not in ignore]    
  return cleaned_text

In [ ]:
def tokenize(sentences):    
  words = [] 
  print(len(sentences))   
  for sentence in sentences:   
    #print((sentence))     
    w = word_extraction(sentence)        
    words.extend(w)            
    words = sorted(list(set(words))) 

  return words

In [ ]:
words = tokenize(sentences)

25000


In [ ]:
len(words)

20248

In [ ]:
# import numpy
# new_balanced_df = []
# for sentence in sentences:        
#   words_temp = word_extraction(sentence)        
#   bag_vector = numpy.zeros(len(words))        
#   for w in words_temp:            
#     for i,word in enumerate(words):                
#       if word == w:                     
#         bag_vector[i] += 1                            
#   new_balanced_df.append(numpy.array(bag_vector))

In [ ]:
max_classes = len(balanced_df.Labels.unique())

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(new_balanced_df, nums, test_size = 0.1, random_state = 42, stratify =nums)


**Model training**

Split into x test and y **test** **bold text**

In [ ]:
max_classes = len(balanced_df.Labels.unique())

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
result = shuffle(balanced_df)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(balanced_df.Text, balanced_df.NumLabel, test_size = 0.1, random_state = 42, stratify =balanced_df.NumLabel)


In [ ]:
len(X_test)

24632

In [ ]:
new_corpus = []
for i in range(0,len(X_train)):
  new_corpus.append(X_train.iloc[i])
len(new_corpus)


221688

In [ ]:
new_corpus2 = []
for i in range(0,len(X_test)):
  new_corpus2.append(X_test.iloc[i])
len(new_corpus2)

24632

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)

In [ ]:

X = cv.fit_transform(new_corpus).toarray()
X_train = X

In [ ]:
var = cv.fit_transform(new_corpus2).toarray()
X_test = var

In [ ]:
len(X_train[0])

1500

In [ ]:
#X_train2 = X_train
#Y_train2 = Y_train

In [ ]:
X_train2 = X_train[0:100000]
Y_train2 = Y_train[0:100000]

In [ ]:
#X_test2 = X_test
#Y_test2 = Y_test

In [ ]:
X_test2 = X_test[0:10000]
Y_test2 = Y_test[0:10000]

In [ ]:
len(X_train2[0])

1500

**MODEL TRAINING**

SVM

In [ ]:
# import support vector classifier
# "Support Vector Classifier"
from sklearn.svm import SVC
clf = SVC(kernel='linear')
# fitting x samples and y classes
clf.fit(X_train,Y_train)

SVC(kernel='linear')

In [ ]:
Y_pred = clf.predict(X_test)
from sklearn import metrics
print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(Y_test, Y_pred))


ACCURACY OF THE MODEL:  0.09507957128937967


In [ ]:
print(Y_pred)
print(Y_test)

[ 9 10  6 ... 13 11  9]
20079      1
182905    11
173883    10
58362      2
144977     7
          ..
128840     6
190414    12
197173    13
194072    12
93131      4
Name: NumLabel, Length: 24632, dtype: int64


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true= Y_test, y_pred= Y_pred))

              precision    recall  f1-score   support

           0       0.02      0.02      0.02      2000
           1       0.21      0.30      0.25      2000
           2       0.11      0.12      0.12      2000
           3       0.07      0.07      0.07      2000
           4       0.39      0.09      0.15      1863
           5       0.06      0.05      0.05      2000
           6       0.07      0.10      0.08      2000
           7       0.10      0.01      0.01      1501
           8       0.04      0.01      0.01       310
           9       0.05      0.11      0.07      1204
          10       0.03      0.20      0.06       535
          11       0.06      0.04      0.04      1611
          12       0.00      0.00      0.00       615
          13       0.64      0.33      0.43      1462
          14       0.02      0.09      0.04       792
          15       0.02      0.00      0.01       267
          16       0.02      0.00      0.00       867
          17       0.13    

K FOLD 

In [ ]:
from sklearn import metrics, cross
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
clf, X_train, Y_train, cv=5, scoring='f1_macro')

ImportError: ignored

In [ ]:
#results = model_selection.cross_val_score(clf, cv=kfold)

PRECISION SCORE

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=42)
model=RandomForestClassifier(n_estimators=50) 

results = model_selection.cross_val_score(estimator=model,
                                          X=X_test,
                                          y=Y_test,
                                          cv=kfold,
                                          scoring=scoring)